In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
test = pd.read_csv('../Feature-eng/test.csv', index_col=None)
X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]

feature_names = X_test.columns
feature_types = X_test.dtypes

In [3]:
# mult = pd.read_csv('../Feature-eng/train-new.csv', index_col=False)
# X, y = mult.iloc[:, 1:], mult.iloc[:, 0]
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.2, 
#                                                     random_state=123,
#                                                     stratify=y)


In [4]:
# dtrain = xgb.DMatrix(data=X_train.values, label=y_train.values)
# dtest = xgb.DMatrix(data=X_test.values, label=y_test.values)


In [5]:
# read in data
dtrain = xgb.DMatrix('../Feature-eng/train.csv?format=csv&label_column=0')
# dtest = xgb.DMatrix(data=X_test.values, label=y_test.values)
dtest = xgb.DMatrix(data=X_test.values, label=y_test.values)
# specify parameters via map
# make prediction
print(dtrain.num_col())

[16:12:53] 964501x169 matrix with 163000669 entries loaded from ../Feature-eng/train.csv?format=csv&label_column=0
169


In [6]:
param = {'max_depth': 10, 'eta': 0.3, 'silent': 1, 'objective': 'binary:logistic', 
         'min_child_weight': 3.781067239275553, 'colsample_bytree': 0.9149522148929845, 
         'gamma': 1.4399646786455722, 'subsample': 0.9171271673354395
        }

param['eval_metric'] = ['auc', 'aucpr', 'error', 'logloss']
evallist = [(dtrain, 'train'), (dtest, 'eval')]


In [7]:
# param = {'max_depth': 6, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softmax', 'subsample': 0.8, 'num_class': 4}

# param['eval_metric'] = ['merror', 'mlogloss']
# evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [8]:
num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3)


[0]	train-auc:0.97676	train-aucpr:0.980923	train-error:0.066198	train-logloss:0.490451	eval-auc:0.9493	eval-aucpr:0.956198	eval-error:0.108431	eval-logloss:0.516989
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 3 rounds.
[1]	train-auc:0.978886	train-aucpr:0.983019	train-error:0.063118	train-logloss:0.381009	eval-auc:0.950447	eval-aucpr:0.948205	eval-error:0.10907	eval-logloss:0.423322
[2]	train-auc:0.980673	train-aucpr:0.984629	train-error:0.061735	train-logloss:0.309905	eval-auc:0.952105	eval-aucpr:0.956937	eval-error:0.107046	eval-logloss:0.368818
[3]	train-auc:0.981808	train-aucpr:0.985601	train-error:0.060441	train-logloss:0.263659	eval-auc:0.952854	eval-aucpr:0.959827	eval-error:0.10766	eval-logloss:0.334417
[4]	train-auc:0.982525	train-aucpr:0.986128	train-error:0.059607	train-logloss:0.233092	eval-auc:0.952781	eval-aucpr:0.960082	eval-error:0.1079	eval-logloss:0.313611
[5]	train-auc:0.983

In [9]:
preds = bst.predict(dtest)
labels = dtest.get_label()
y_pred = (preds > 0.5).astype(int)
# print(labels[0:10])

In [19]:
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(labels, y_pred))
print('Accuracy {}'.format(accuracy_score(labels, y_pred)))

              precision    recall  f1-score   support

         0.0       0.85      0.95      0.90     59868
         1.0       0.95      0.83      0.89     60669

   micro avg       0.89      0.89      0.89    120537
   macro avg       0.90      0.89      0.89    120537
weighted avg       0.90      0.89      0.89    120537

Accuracy 0.8917593767888699


In [11]:
# from sklearn.metrics import log_loss
# from sklearn.metrics import classification_report
# print(classification_report(labels, preds))

In [20]:
print(log_loss(labels.astype('float64'), preds.astype('float64'), normalize=True, eps=1e-15))

0.2814141846764489


In [13]:
# import matplotlib.pyplot as plt
# fig = plt.figure(figsize=(100,100))
# axes = fig.add_axes(rect=[1,1,1,1])
# xgb.plot_importance(bst, axes)

In [14]:
# print(log_loss(labels, bst.predict(dtest), normalize=True))

In [15]:
# bst.dump_model('xgb-baseline.txt')

In [16]:
sorted_x = sorted(bst.get_fscore().items(), key=lambda kv: kv[1])
print(sorted_x)

[('f39', 1), ('f168', 1), ('f61', 1), ('f127', 1), ('f113', 1), ('f54', 1), ('f87', 2), ('f37', 2), ('f80', 2), ('f108', 2), ('f117', 2), ('f26', 2), ('f88', 3), ('f19', 3), ('f123', 3), ('f81', 3), ('f72', 3), ('f35', 3), ('f52', 3), ('f126', 3), ('f155', 3), ('f141', 3), ('f86', 4), ('f148', 4), ('f79', 4), ('f58', 5), ('f91', 6), ('f102', 6), ('f30', 6), ('f59', 6), ('f125', 7), ('f112', 7), ('f82', 7), ('f49', 8), ('f119', 9), ('f128', 9), ('f134', 9), ('f42', 10), ('f83', 10), ('f38', 11), ('f104', 11), ('f47', 11), ('f92', 11), ('f110', 13), ('f100', 14), ('f70', 14), ('f63', 14), ('f111', 15), ('f132', 16), ('f46', 16), ('f115', 16), ('f120', 17), ('f71', 17), ('f130', 18), ('f84', 18), ('f44', 18), ('f69', 19), ('f68', 20), ('f18', 21), ('f109', 26), ('f103', 26), ('f51', 26), ('f36', 27), ('f62', 27), ('f129', 28), ('f98', 30), ('f29', 32), ('f75', 33), ('f85', 33), ('f77', 37), ('f5', 44), ('f78', 44), ('f16', 46), ('f20', 48), ('f31', 52), ('f24', 61), ('f15', 69), ('f10', 7

In [18]:
print(feature_names[28])
print(feature_names[0])
print(feature_names[6])
print(feature_names[7])
print(feature_names[13])
print(feature_names[17])

AVG_TIME_TO_PAYMENT
BILLING_PLAN_ID
CREATED
SCHEDULED
TOTALPAYMENT_COUNT
CC_AMOUNT
